## Подготовка данных для занятия

In [91]:
import re

import pandas as pd
import pymorphy2


text_df = pd.read_csv("content_description.csv", sep='\t')
text_df.head()

# разбиваем текст на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)
# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
tokens_df.groupby(['word', 'doc_id'])['dummy'].count().reset_index().sort_values(by='dummy', ascending=False)
word_count_df.sort_values(by='dummy', ascending=False).head(10)



,doc_id,word,dummy
6,0,в,10
731,7,и,9
610,6,и,9
583,6,в,8
313,3,и,8
144,1,и,8
29,0,и,7
173,1,он,6
400,4,в,6
454,4,на,6


# Домашняя работа

1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процент документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите, какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [66]:
new_tokens_df = tokens_df.groupby(['word', 'doc_id'])['dummy'].count().reset_index().sort_values(by='dummy', ascending=False)
new_tokens_df['dummy'] = 1

doc_frequency = new_tokens_df.groupby('word')['dummy'].count().reset_index().sort_values(by='dummy', ascending=False)
doc_frequency['doc_frequency'] = round(doc_frequency['dummy'] / (new_tokens_df['doc_id'].max() + 1) * 100)
doc_frequency.drop('dummy', axis=1, inplace=True)

doc_frequency = doc_frequency[doc_frequency['word'].str.len() > 3]
doc_frequency.head()


,word,doc_frequency
480,свой,75.0
326,онлайн,75.0
498,смотреть,62.0
483,себя,50.0
471,самый,50.0


2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [85]:
raw_text = text_df.description.values[3]
# регулярка - её нужно поправить
reg_expr = r'[А-Я][а-я]{2,10}\s[А-Я][а-я]*'
# компилируем регулярное выражение
reg_expr_compiled = re.compile(reg_expr)
# применяем выражение к тексту

for i in reg_expr_compiled.findall(raw_text)[:3]:
    print(i)


Стивена Фрая
Джоном Дженксом
Теда Уоллеса


3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а также таблицей `word_count_df`, сформируйте таблицу c двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`
* соединить полученный датафрейм с помощью функции `merge` с ранее полученным датафреймом `word_count_df`, который содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [94]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}

nested_genres = [[(i, j) for j in genre_dict[i]] for i in genre_dict]
# переводим из словаря к более удобному виду
flatten_genres = list(itertools.chain(*nested_genres))
# создаём DataFrame
genres_df = pd.DataFrame(flatten_genres, columns = ['genre', 'word'])

genres_df.merge(word_count_df, how='inner', on='word').sort_values(by='doc_id')

,genre,word,doc_id,dummy
6,детектив,тайна,0,1
7,детектив,разгадать,0,1
8,детектив,загадочный,0,1
9,триллер,ужас,0,1
5,сказка,милый,1,1
3,мелодрама,выбор,2,1
4,мелодрама,позор,2,1
0,комедия,сатирический,3,1
1,комедия,авантюрный,4,1
2,комедия,забавный,4,1


* **Текст 0**: детектив, триллер;
* **Текст 1**: сказка;
* **Текст 2**: мелодрама;
* **Текст 3**: комедия;
* **Текст 4**: комедия;
* **Текст 6**: триллер.